# Lecture 4.2: Time Evolution and Quantum Quenches with TEBD

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/4-DMRG-TEBD/6c.TEBD_XXZ_quench.ipynb)

In this notebook, you will study **quantum quench dynamics** in the **XXZ Heisenberg chain** using **Time-Evolving Block Decimation (TEBD)**.

The scientific goal is to study correlation spreading after a "quantum quench" and observe the growth of entanglement entropy over time, and understand why dynamics is so much more challenging than ground state calculation.
Ideally you wil also be able to identify the light-cone structure in correlation propagation

## The Model: XXZ Heisenberg Chain

We will study the **1D spin-1/2 XXZ chain** with **open boundary conditions**:

$$
H = J \sum_{i=1}^{L-1} \left[ S_i^x S_{i+1}^x + S_i^y S_{i+1}^y + \Delta S_i^z S_{i+1}^z \right] - h \sum_{i=1}^{L} S_i^z
$$

where $S^{x,y,z}$ are spin-1/2 operators (with eigenvalues $\pm 1/2$).

### Key Parameters

- **$\Delta$**: Anisotropy parameter
  - $\Delta = 0$: XY model (free fermions)
  - $\Delta = 1$: Heisenberg model (SU(2) symmetric)
  - $\Delta > 1$: Easy-axis (Ising-like)
  - $\Delta < 1$: Easy-plane (XY-like)
- **$h$**: Magnetic field (breaks $S^z$ conservation)
- **$J$**: Exchange coupling (we typically set $J=1$)

### Quantum Quench Protocol

To quench a system means to rapidly change its configuration. In Quantum Mechanics, we refer to quenches when we prepare the ground-state of a system and then evolve them with a different Hamiltonian. 
To simulate quenches, we need to follow those four points:

1. **Prepare initial state**: Find the ground state of $H(\Delta_i, h_i)$ (using DMRG or other algorithms)
2. **Time evolution**: Evolve under $H(\Delta_f, h_f)$ (using TEBD or alternatives)

And in general we will track correlations, magnetization, entanglement along the dynamics. We don't need to compute those at each microscopic small timestep, but we can do it every once in a while with a reasonable resolution.

# Packages

For this notebook, you will use **TeNPy** (Tensor Network Python), a powerful library for tensor network algorithms.
- Main docs: https://tenpy.readthedocs.io
- TEBD tutorial: https://tenpy.readthedocs.io/en/stable/notebooks/00_tebd.html
- XXZ model: https://tenpy.readthedocs.io/en/stable/reference/tenpy.networks.mps.MPS.html

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import scipy.linalg

# TeNPy imports
import tenpy
from tenpy.networks.mps import MPS
from tenpy.models.xxz_chain import XXZChain
from tenpy.algorithms import dmrg
from tenpy.algorithms import tebd

# Nicer plots
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

---

# 1. Prepare Initial Ground State with DMRG

Before studying dynamics, you need an initial state. Use DMRG to find the ground state of the XXZ chain.

## Setup

**Initial Hamiltonian parameters:**
- $L = 40$ (system size)
- $\Delta_i = 0.5$ (easy-plane regime, gapless)
- $h_i = 0.0$ (no field initially)
- $J = 1.0$ (exchange coupling)
- Use **open boundary conditions**

## Instructions

1. Define the XXZ model using `XXZChain` with the parameters above
   - Use `XXZChain(model_params)` where `model_params` is a dictionary
   - Key parameters: `L`, `Jxx`, `Jz`, `hz`, `bc_MPS='finite'`
   - Note: TeNPy uses `Jxx` for $J$ and `Jz` for $J\Delta$, and $S^z$ has eigenvalues $\pm 1/2$

2. Create an initial MPS in a product state
   - Use `MPS.from_product_state(...)` with `"up"` (all spins up) or `"down"`
   - Alternative: use `MPS.from_lat_product_state(model.lat, ["up"])` 

3. Run DMRG to find the ground state
   - Use `dmrg.run(psi, model, dmrg_params)`
   - Important DMRG parameters:
     - `'trunc_params': {'chi_max': 100, 'svd_min': 1e-10}` (bond dimension and cutoff)
     - `'max_E_err': 1e-10` (energy convergence criterion)
     - `'max_sweeps': 20` (maximum number of sweeps)

4. Print the ground state energy and check convergence by plotting the energy curve
   - The `dmrg.run()` returns a dictionary with `'E'` (energy) and other info

## TODO:

- Verify that DMRG converged (check the output or convergence messages)
- Print the final energy per site: $E_0 / L$
- Print the maximum bond dimension $\chi$ reached in the MPS
   - Use `psi.chi` to get bond dimensions
   - For gapless systems, you should see substantial entanglement even in the ground state

In [ ]:
# TODO: Implement Task 1
#
# 1. Define model parameters for initial state
# 2. Create XXZChain model
# 3. Create initial product state MPS  
# 4. Run DMRG
# 5. Print results

# Your code here


---

# Task 2: Perform Quantum Quench and Time Evolution

Now perform the quantum quench and evolve the system forward in time.

## Quench Protocol

**Final Hamiltonian parameters:**
- $\Delta_f = 2.0$ (easy-axis regime, gapped)
- $h_f = 0.5$ (apply magnetic field)
- Keep $J = 1.0$, $L = 40$

**Time evolution parameters:**
- Total time: $t_{\text{max}} = 5.0$ (in units of $J^{-1}$)
- Time step: $dt = 0.1$ (Trotter step)
- Maximum bond dimension: $\chi_{\text{max}} = 100$ (increase if needed)
- Measurement interval: measure every $N = 1$ time step (adjust for performance)

## Instructions

1. **Create the quench Hamiltonian**
   - Define new `XXZChain` with $(\Delta_f, h_f)$
   - This is the Hamiltonian you'll evolve under

2. **Setup TEBD engine**
   - Use `tebd.TEBDEngine(psi, model, tebd_params)`
   - Important TEBD parameters:
     - `'order': 2` (use 2nd order Trotter decomposition for better accuracy)
     - `'dt': 0.1` (time step)
     - `'N_steps': 1` (steps between measurements)
     - `'trunc_params': {'chi_max': 100, 'svd_min': 1e-10}`

3. **Time evolution loop**
   - Evolve for $n_{\text{steps}} = t_{\text{max}}/dt$ steps
   - At each measurement step, record:
     - **Time** $t$
     - **Energy**: use `model.H_MPO.expectation_value(psi)`
     - **Magnetization** $\langle S^z_i \rangle$ at each site: use `psi.expectation_value("Sz")`
     - **Maximum bond dimension**: `max(psi.chi)`
     - **Entanglement entropy**: `psi.entanglement_entropy()` at the center bond
   - Use `eng.run()` or `eng.run_evolution(N_steps=1)` to evolve by one step

4. **Store data in arrays**
   - Create lists or arrays for times, energies, magnetizations (shape: `[n_times, L]`), etc.
   - This will make plotting easier

## Tips

- Start with shorter evolution time or larger $dt$ for testing, but then show that the dt you have is good enough by reducing it and showing that the dynamics converges to the same curve.
- Monitor bond dimension growth, and if it hits $\chi_{\text{max}}$, you need to icnrease it. At some point, it will be impossible to increase it further. That's ok, but make it explicit. You hshould also in this case show that the dynamics converge as a function of $\chi$. 
- TEBD can be slow for large $L$ and small $dt$... It can help to reduce the frequency at which you compute observables (i think)

In [ ]:
# TODO: Implement Task 2
#
# 1. Create quench Hamiltonian with new parameters
# 2. Setup TEBD engine
# 3. Create storage arrays for observables
# 4. Run evolution loop with measurements
# 5. Print progress occasionally

# Your code here


---

# 2. Analyze Quench Dynamics

Now visualize and analyze the time evolution data.

## 2a. Energy evolution: 

Plot the energy against time $E(t)$
- Add a **horizontal dashed line** showing the initial energy $E_0$ (ground state before quench)
- Comment what you see.
- Ideally, try to put in the same plot the energy computed for the same quench using different values of bond dimension truncation $\chi_{max}$. What do you observe?

In [ ]:
# TODO: Plot energy vs time
# Include initial energy as reference line


## 2b. Magnetization Profile: 

Create a **2D color plot** (heatmap) showing how magnetization evolves in space and time.
On the x axis you should have site indices, on the y axis the time, and the color should signal the magnetization (please add the colorbar!)

Do this plot for different quenches (for different values of final field). Interpret.

In [ ]:
# TODO: Create 2D heatmap of magnetization


## 2c. Entanglement Entropy Growth

Plot $S(t)$ for the **center bond** (bond $L/2$) vs time to see how it grows. 

You should notice that:
- Entanglement entropy typically **grows linearly** at early times after a quench
- Growth rate is related to entanglement velocity (often close to maximum group velocity)
- Eventually saturates or hits $\chi_{\text{max}}$ limit

Try to:
- Fit the linear regime to extract entanglement growth rate $dS/dt$
- Print this value
- Compare to typical energy scale $J$ - what are the units?

In [ ]:
# TODO: Plot entanglement entropy vs time


## 2d. Bond Dimension Evolution  

Plot maximum bond dimension $\chi_{\text{max}}(t)$ vs time:

- Shows how much entanglement is created by the quench
- If $\chi$ saturates at your `chi_max` limit, you're **truncating** - results may be inaccurate!
- **What to check:**
  - Does $\chi$ grow exponentially, linearly, or saturate?
  - Is it bounded below your `chi_max` setting?

Add a horizontal line showing your `chi_max` parameter for reference.

In [ ]:
# TODO: Plot bond dimension vs time
# Add horizontal line at chi_max limit

# Your code here


---

# 3. Correlation Function Spreading

A key signature of quench dynamics is the **spreading of correlations**. Compute and visualize the connected correlation function, which is

$$
C(i,j,t) = \langle S^z_i(t) S^z_j(t) \rangle - \langle S^z_i(t) \rangle \langle S^z_j(t) \rangle
$$

After a quench, correlations spread from initially correlated regions, forming a **light cone** structure.


During your time evolution (or in a new evolution run), measure:

1. **Two-point correlator** $\langle S^z_i S^z_j \rangle$. Use `psi.correlation_function("Sz", "Sz")` which returns matrix $C[i,j]$, which gives $\langle S^z_i S^z_j \rangle$ for all pairs
2. **One-point expectation** $\langle S^z_i \rangle$ with `psi.expectation_value("Sz")`
3. **Compute connected correlator:** by combining the two items above

In [ ]:
# TODO: Measure correlations during evolution
#


### Optional: Visualize the Light Cone Structure

Create a **2D heatmap** showing correlation spreading. Fix a reference site $i_0 = L/2$ (center), and plot the correlations of this point with the other sites. Plot $C_{\text{conn}}(i_0, j, t)$ as function of Site separation $r = |j - i_0|$  (x axis), time (y axis).

You should observe:
- **Light cone edges:** Correlation should be zero outside a cone $|r| < v_{\text{max}} \cdot t$
- $v_{\text{max}}$ is the Lieb-Robinson velocity (maximum information spreading speed)
- Inside the cone there will be complex interference patterns, while outside there will be nothing.

Draw dashed lines indicating the theoretical Lieb-Robinson bound (for XXZ, $v_{\text{LR}} \approx 2J$ for $|\Delta| \leq 1$). 

In [ ]:
# TODO: Create light cone plot


### Optional: Extract Correlation Spreading Velocity

Estimate the spreading velocity from your light cone plot:

1. For each time, find the **maximum distance** $r_{\text{max}}(t)$ where correlations are non-zero (above some threshold)
2. Plot $r_{\text{max}}$ vs $t$
3. Fit to $r_{\text{max}} = v \cdot t$ to extract velocity $v$
4. Compare to theoretical expectations:
   - For XXZ chain: $v \sim J \cdot \min(1, |\Delta|)$ (approximate)

Print the extracted velocity with units.

In [ ]:
# TODO: Extract and fit spreading velocity
#
# 1. Determine correlation front position at each time
# 2. Linear fit
# 3. Plot and print velocity

# Your code here
